# Set up the environment

In [2]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [3]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


**Install Transformers (From hugging face)**


In [4]:
!pip install transformers;

In [5]:
#!wget http://nlp.stanford.edu/data/glove.twitter.27B.zip
#!unzip /content/glove.twitter.27B.zip

# Parse

In [6]:
import pandas as pd

In [7]:
url = '/content/drive/My Drive/df_labeled.csv'
df = pd.read_csv(url)

In [8]:
df.head()

,id,tweet,field,brexit,censorship,corruption,crisis,economy,education,energies,environment,ep members,ep performance,ep shape,eu elections,eu members,eu shape,exterior,fifa,google,hack,haters,health,human rights,incidents,inmigration,jokes,meps,news,opinion,policies,political side,privacy,requests,security,speeches,technology,terrorism,trade,working rights
0,220484136727621600,ACTA Left For Dead by European Parliament: It ...,policies,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,11746497168,European Parliament delegation to observe elec...,exterior,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1039549618391064960,Hungary's Orban vows to defy European Parliame...,meps,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,316120129341620224,Who represents us in the European Parliament? ...,meps,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,845293933919965184,European Parliament calls for legal rules on r...,policies,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [178]:
df = df[['tweet','field']]
df.dropna(inplace=True)
topic = 'brexit'
dummies = pd.get_dummies(df["field"])
dummies_topic = pd.DataFrame(dummies[topic])
df_topic = pd.concat([df.tweet, dummies_topic], axis=1)

In [179]:
dummies.head()

,brexit,censorship,corruption,crisis,economy,education,energies,environment,ep members,ep performance,ep shape,eu elections,eu members,eu shape,exterior,fifa,google,hack,haters,health,human rights,incidents,inmigration,jokes,meps,news,opinion,policies,political side,privacy,requests,security,speeches,technology,terrorism,trade,working rights
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [180]:
df.head()

,tweet,field
0,ACTA Left For Dead by European Parliament: It ...,policies
1,European Parliament delegation to observe elec...,exterior
2,Hungary's Orban vows to defy European Parliame...,meps
3,Who represents us in the European Parliament? ...,meps
4,European Parliament calls for legal rules on r...,policies


In [181]:
# Get the lists
sentences = df_topic.tweet.values
labels = df_topic[topic].values

# Tokenization

### Bert Tokenizer

In [182]:
from transformers import BertTokenizer
# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [183]:
# Print the original sentence.
print(' Original: ', sentences[20])

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(sentences[20]))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[20])))

 Original:  European Parliament report accuses Wilbur Ross of insider trading  http://www.motherjones.com/politics/2017/12/european-parliament-report-accuses-wilbur-ross-of-insider-trading/ … via @MotherJones
Tokenized:  ['european', 'parliament', 'report', 'accuse', '##s', 'wilbur', 'ross', 'of', 'insider', 'trading', 'http', ':', '/', '/', 'www', '.', 'mother', '##jon', '##es', '.', 'com', '/', 'politics', '/', '2017', '/', '12', '/', 'european', '-', 'parliament', '-', 'report', '-', 'accuse', '##s', '-', 'wilbur', '-', 'ross', '-', 'of', '-', 'insider', '-', 'trading', '/', '…', 'via', '@', 'mother', '##jon', '##es']
Token IDs:  [2647, 3323, 3189, 26960, 2015, 26151, 5811, 1997, 25297, 6202, 8299, 1024, 1013, 1013, 7479, 1012, 2388, 14339, 2229, 1012, 4012, 1013, 4331, 1013, 2418, 1013, 2260, 1013, 2647, 1011, 3323, 1011, 3189, 1011, 26960, 2015, 1011, 26151, 1011, 5811, 1011, 1997, 1011, 25297, 1011, 6202, 1013, 1529, 3081, 1030, 2388, 14339, 2229]


In [184]:
max_len = 0

# For every sentence...
for sent in sentences:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

Max sentence length:  169


## Tokenize dataset

In [185]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in sentences:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        truncation=True,
                        max_length = 200,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

Original:  ACTA Left For Dead by European Parliament: It was loathed by anyone with a vested interest in ...  http://bit.ly/MWuWdG  By @ubuntuprecise
Token IDs: tensor([  101,  2552,  2050,  2187,  2005,  2757,  2011,  2647,  3323,  1024,
         2009,  2001,  8840,  8988,  2098,  2011,  3087,  2007,  1037, 26003,
         3037,  1999,  1012,  1012,  1012,  8299,  1024,  1013,  1013,  2978,
         1012,  1048,  2100,  1013, 12464, 25974,  2094,  2290,  2011,  1030,
         1057,  8569,  3372,  6279,  2890, 18380,   102,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,   

## Training and validation split

In [186]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels)

# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

  632 training samples
  158 validation samples


In [187]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [188]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [189]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30522, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

### Optimizer and Learning Rate Scheduler

In [190]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

In [191]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 4

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

## Training loop

In [192]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [193]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [194]:
import torch
torch.cuda.empty_cache()

In [195]:
import random
import numpy as np

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device).float()
        b_labels = batch[2].to(device).type(torch.cuda.LongTensor)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        loss, logits = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)
        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device).type(torch.cuda.LongTensor)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            (loss, logits) = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 4 ========
Training...

  Average training loss: 0.46
  Training epcoh took: 0:00:12

Running Validation...
  Accuracy: 0.90
  Validation Loss: 0.32
  Validation took: 0:00:01

======== Epoch 2 / 4 ========
Training...

  Average training loss: 0.37
  Training epcoh took: 0:00:12

Running Validation...
  Accuracy: 0.90
  Validation Loss: 0.32
  Validation took: 0:00:01

======== Epoch 3 / 4 ========
Training...

  Average training loss: 0.35
  Training epcoh took: 0:00:12

Running Validation...
  Accuracy: 0.90
  Validation Loss: 0.28
  Validation took: 0:00:01

======== Epoch 4 / 4 ========
Training...

  Average training loss: 0.28
  Training epcoh took: 0:00:12

Running Validation...
  Accuracy: 0.90
  Validation Loss: 0.21
  Validation took: 0:00:01

Training complete!
Total training took 0:00:52 (h:mm:ss)


# LABEL OTHER TWEETS

## Parse

In [196]:
# Load the dataset into a pandas dataframe.
df_full = pd.read_excel(r'/content/drive/My Drive/df_full_august.xlsx')

In [197]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 972564 entries, 0 to 972563
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  972564 non-null  int64 
 1   id          972564 non-null  int64 
 2   date        972564 non-null  object
 3   tweet       972564 non-null  object
dtypes: int64(2), object(2)
memory usage: 29.7+ MB


In [198]:
df_full.sample(5)

,Unnamed: 0,id,date,tweet
334015,114379,1065222519802797952,2018-11-21,European Parliament: Stop the censorship-machi...
439608,37409,608971457650130944,2015-06-11,The European Parliament has called for Sepp Bl...
481360,79161,660012154112704512,2015-10-30,EU wants asylum for Snowden: The European Parl...
463694,61495,636138224918753280,2015-08-25,Over 60 reps of European Youth Parliament visi...
281111,61475,1014564083776007936,2018-07-04,Tories enter into alliance with Swedish far-ri...


In [199]:
df_full.tweet.isna().sum(), df_full.shape

(0, (972564, 4))

In [200]:
df_full = df_full[['id','tweet']]

In [201]:
df_full["tweet"] = df_full["tweet"].astype(str)

In [202]:
#df_full = df_full.sample(1000)

In [203]:
df_full.head()

,id,tweet
0,1080230965912449024,The European elections next May will give an o...
1,1080230342324301952,European Parliament: Stop the censorship-machi...
2,1080229977730280960,Purlease. \n\nThe European Parliament is a pre...
3,1080229425206231040,European Parliament: Stop the censorship-machi...
4,1080228717358731008,"Our last year in the EU, if Theresa May gets h..."


In [204]:
# Get the lists
sentences = df_full.tweet.values

## Tokenize full dataset

In [205]:
# Print the original sentence.
print(' Original: ', sentences[20])

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(sentences[20]))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[20])))

 Original:  Ask the European Parliament to Stop Serving Foie Gras  http://petauk.org/eufoiegras  #banfoiegras @TheSun @Telegraph #foiegras @Conservatives
Tokenized:  ['ask', 'the', 'european', 'parliament', 'to', 'stop', 'serving', 'f', '##oi', '##e', 'gr', '##as', 'http', ':', '/', '/', 'pet', '##au', '##k', '.', 'org', '/', 'eu', '##fo', '##ieg', '##ras', '#', 'ban', '##fo', '##ieg', '##ras', '@', 'the', '##sun', '@', 'telegraph', '#', 'f', '##oi', '##eg', '##ras', '@', 'conservatives']
Token IDs:  [3198, 1996, 2647, 3323, 2000, 2644, 3529, 1042, 10448, 2063, 24665, 3022, 8299, 1024, 1013, 1013, 9004, 4887, 2243, 1012, 8917, 1013, 7327, 14876, 28872, 8180, 1001, 7221, 14876, 28872, 8180, 1030, 1996, 19729, 1030, 10013, 1001, 1042, 10448, 13910, 8180, 1030, 11992]


In [206]:
from transformers import BertTokenizer
# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [207]:
sentences[1]

'European Parliament: Stop the censorship-machinery! Save the Internet! - Sign the Petition!  http://chng.it/6mSYYcPs\xa0 via @Change'

In [208]:
max_len = 0

# For every sentence...
for sent in sentences:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

Token indices sequence length is longer than the specified maximum sequence length for this model (624 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (613 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (590 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (562 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1523 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for th

Max sentence length:  1525


In [209]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in sentences:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        truncation=True,
                        max_length = 200,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

Original:  The European elections next May will give an opportunity to all the people of the Nations of Europe to send a modern nationalist majority at the European Parliament. #Europeennes2019 https://twitter.com/JackPosobiec/status/1080220614223380480 …
Token IDs: tensor([  101,  1996,  2647,  3864,  2279,  2089,  2097,  2507,  2019,  4495,
         2000,  2035,  1996,  2111,  1997,  1996,  3741,  1997,  2885,  2000,
         4604,  1037,  2715,  8986,  3484,  2012,  1996,  2647,  3323,  1012,
         1001,  2885, 24336,  2015, 11387, 16147, 16770,  1024,  1013,  1013,
        10474,  1012,  4012,  1013,  2990,  6873,  6499, 11283,  2278,  1013,
         3570,  1013, 10715,  2692, 19317,  2692,  2575, 16932, 19317, 22394,
        17914, 18139,  2692,  1529,   102,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,   

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [210]:
# Convert to tensors.
prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
# prediction_labels = torch.tensor(labels)

# Set the batch size.  
batch_size = 16  

# Create the DataLoader.
prediction_data = TensorDataset(prediction_inputs, prediction_masks)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
# Prediction on test set

print('Predicting labels for {:,} test sentences...'.format(len(prediction_inputs)))

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions  = []

# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask = batch
  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)

  logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  # label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  # true_labels.append(label_ids)

print('    DONE.')


Predicting labels for 972,564 test sentences...


In [168]:
# Combine the predictions for each batch into a single list of 0s and 1s.
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()

In [169]:
table1 = pd.DataFrame(flat_predictions)
df1 = df_full[['id', 'tweet']]
table1.reset_index(drop=True, inplace=True)
df1.reset_index(drop=True, inplace=True)
pred_output1 = pd.concat([df1,table1], axis=1)

In [170]:
pred_output1.shape

(972564, 3)

In [171]:
pred_output1.sample(5)

,id,tweet,0
725231,220931693790507008,European Parliament Rejects Anti-Piracy Treaty...,0
629805,13835422732,I think I'm the only young adult in America ke...,0
714732,213226973038784512,I Want a Job and I Want it Now480: The S&D Gro...,0
579234,796713513632862208,European Parliament considers plan to let indi...,0
275249,1009372217996726016,"Korrejt 👉 ""On 20 June, the Legal Affairs Commi...",0


In [172]:
table1.shape, df1.shape, pred_output1.shape

((972564, 1), (972564, 2), (972564, 3))

In [173]:
pred_output1.isna().sum()

id       0
tweet    0
0        0
dtype: int64

In [174]:
pred_output1.columns = ['id', 'tweet', 'topic']

In [176]:
pred_output1.topic[pred_output1['topic']==1].sum()

0

In [175]:
pred_output1.to_csv('environment_labeled.csv', index=True)

In [ ]:
#pred_output1[pred_output1.topic==1].sum()

# Analyzing topics

## Brexit

In [105]:
pd.read_csv('/content/brexit_labeled.csv', lineterminator='\n').head(5)

,Unnamed: 0,id,tweet,topic
0,0,1080230965912449024,The European elections next May will give an o...,0
1,1,1080230342324301952,European Parliament: Stop the censorship-machi...,1
2,2,1080229977730280960,Purlease. \n\nThe European Parliament is a pre...,0
3,3,1080229425206231040,European Parliament: Stop the censorship-machi...,1
4,4,1080228717358731008,"Our last year in the EU, if Theresa May gets h...",0


## Censorship

In [ ]:
pd.read_csv('/content/censorship_labeled.csv', lineterminator='\n').head(5)

Exterior

In [88]:
pd.read_csv('/content/exterior_labeled.csv', lineterminator='\n').sample(10)

,Unnamed: 0,id,tweet,topic
640761,640761,28818905013,(Current) MEPs turn new clout on fiscal reform...,0
213983,213983,1204392070175239936,European Parliament getting ready for our Thur...,0
590023,590023,809411500225994752,Key European Parliament committee moves to boo...,0
834321,834321,425306353004212224,Licia Ronzulli is an Italian member of the Eur...,0
658125,658125,67887490744332288,BREAKING NEWS: European Parliament urges EU Co...,0
185732,185732,1176820648343032064,"Well they are following in Nazi footsteps, usi...",0
734777,734777,252311108516470784,@PaulMaskeyMP Paul what would be the proper ch...,0
471609,471609,652005514419798016,"Angela Merkel -European Parliament: ""The refug...",0
779951,779951,324501091817488384,Lawmaker: EU must push for Cyprus' reunificati...,0
326959,326959,1062024859624960000,The Vice President of the European Parliament ...,0
